In [1]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

In [2]:
from utils import create_all_folders, Folders

create_all_folders()

In [3]:
from layers import AMF_GD_YOLOv8
import torch
from data_processing import ImageData
from training_parameters import (
    class_names,
    class_indices,
    class_colors,
    transform_pixel_rgb,
    transform_spatial,
    proba_drop_rgb,
    proba_drop_chm,
    labels_transformation_drop_chm,
    labels_transformation_drop_rgb,
)
from training import (
    train_and_validate,
    create_and_save_splitted_datasets,
    load_tree_datasets_from_split,
)
import multiprocessing as mp

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "garbage_collection_threshold:0.6,max_split_size_mb:512"
print(os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))

garbage_collection_threshold:0.6,max_split_size_mb:512


In [5]:
# Find and split the data

full_image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
resolution = 0.08

image_data = ImageData(full_image_path)

annotations_folder_path = os.path.join(Folders.CROPPED_ANNOTS.value, image_data.base_name)
rgb_folder_path = os.path.join(Folders.CROPPED_IMAGES.value, image_data.base_name)
chm_folder_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "cropped",
    image_data.coord_name,
)

sets_ratios = [3, 1, 1]
sets_names = ["training", "validation", "test"]
data_split_file_path = "../data/others/data_split.json"
dismissed_classes = []

create_and_save_splitted_datasets(
    rgb_folder_path,
    chm_folder_path,
    annotations_folder_path,
    sets_ratios,
    sets_names,
    data_split_file_path,
    random_seed=0,
)

datasets = load_tree_datasets_from_split(
    data_split_file_path,
    class_indices,
    class_colors,
    proba_drop_rgb=proba_drop_rgb,
    labels_transformation_drop_rgb=labels_transformation_drop_rgb,
    proba_drop_chm=proba_drop_chm,
    labels_transformation_drop_chm=labels_transformation_drop_chm,
    dismissed_classes=dismissed_classes,
    transform_spatial_training=transform_spatial,
    transform_pixel_training=transform_pixel_rgb,
)

# Training parameters

lr = 1e-2
epochs = 800

batch_size = 6
num_workers = mp.cpu_count()
accumulate = 12

In [6]:
index = 0
model_name = f"trained_model_{epochs}ep_{index}"
while os.path.exists(f"../models/amf_gd_yolov8/{model_name}.pt"):
    index += 1
    model_name = f"trained_model_{epochs}ep_{index}"

model = AMF_GD_YOLOv8(
    3, 1, device=device, scale="n", class_names=class_names, name=model_name
).to(device)

final_model = train_and_validate(
    model=model,
    datasets=datasets,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    accumulate=accumulate,
    device=device,
    save_outputs=True,
)
state_dict = final_model.state_dict()
torch.save(state_dict, f"../models/amf_gd_yolov8/{model_name}.pt")

Output()

/home/alexandre/miniforge3/envs/tree-segment/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1716579264291/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/alexandre/miniforge3/envs/tree-segment/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1716579264291/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [ ]:
# from time import time
# import multiprocessing as mp
# from training import TreeDataLoader

# batch_size = 8

# for num_workers in range(2, mp.cpu_count() + 2, 2):
#     train_loader = TreeDataLoader(
#         datasets["training"],
#         batch_size=batch_size,
#         shuffle=True,
#         num_workers=num_workers,
#         pin_memory=True,
#     )
#     start = time()
#     for epoch in range(1, 3):
#         for i, data in enumerate(train_loader, 0):
#             pass
#     end = time()
#     print(f"Finish with: {end - start} second, num_workers={num_workers}")

Finish with: 9.32326626777649 second, num_workers=2
Finish with: 6.183119773864746 second, num_workers=4
Finish with: 5.941167593002319 second, num_workers=6
Finish with: 5.7705371379852295 second, num_workers=8


In [6]:
from training import test_save_output_image, initialize_dataloaders

index = 0
model_name = f"trained_model_{epochs}ep_{index}"

model = AMF_GD_YOLOv8(
    3, 1, device=device, scale="n", class_names=class_names, name=model_name
).to(device)

state_dict = torch.load(f"../models/amf_gd_yolov8/{model_name}.pt")
model.load_state_dict(state_dict)

_, _, test_loader = initialize_dataloaders(
    datasets=datasets, batch_size=batch_size, num_workers=num_workers
)

test_save_output_image(
    model,
    test_loader,
    -1,
    device,
    no_rgb=False,
    no_chm=False,
    save_path=f"../data/others/model_output/{model_name}_all.geojson",
)

test_save_output_image(
    model,
    test_loader,
    -1,
    device,
    no_rgb=True,
    no_chm=False,
    save_path=f"../data/others/model_output/{model_name}_no_rgb.geojson",
)
test_save_output_image(
    model,
    test_loader,
    -1,
    device,
    no_rgb=False,
    no_chm=True,
    save_path=f"../data/others/model_output/{model_name}_no_chm.geojson",
)
test_save_output_image(
    model,
    test_loader,
    -1,
    device,
    no_rgb=True,
    no_chm=True,
    save_path=f"../data/others/model_output/{model_name}_no_rgb_no_chm.geojson",
)